In [ ]:
!pip install numpy pandas scikit-learn tensorflow keras scikit-fuzzy imbalanced-learn

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/anfis-0.3.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/scikit_fuzzy-0.5.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
!pip install -U scikit-fuzzy

DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/anfis-0.3.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.11/dist-packages/scikit_fuzzy-0.5.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv("fraud_data.csv")  # Replace with your actual dataset file

# Drop irrelevant columns (Modify based on your dataset)
df.drop(["Dummy Policy No", "CORRESPONDENCEPOSTCODE", "POLICYRISKCOMMENCEMENTDATE", "Date of Death", "INTIMATIONDATE", "Bank code", "Policy Payment Term", "CORRESPONDENCECITY", "CORRESPONDENCESTATE"], axis=1, inplace=True, errors='ignore')

# Convert all string data in the dataset to lowercase
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Handling missing values
df.fillna(df.median(numeric_only=True), inplace=True)

df.head()

<ipython-input-307-6ec7b7c44548>:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,ASSURED_AGE,NOMINEE_RELATION,OCCUPATION,POLICY SUMASSURED,Premium,PREMIUMPAYMENTMODE,Annual Income,HOLDERMARITALSTATUS,INDIV_REQUIREMENTFLAG,Policy Term,Product Type,CHANNEL,STATUS,SUB_STATUS,Fraud Category
0,20,mother,service,"1,200,000","120,000",quarterly,"420,000",single,non medical,10,ulip,retail agency,claim,death claim repudiated,misrepresentation
1,33,husband,service,"350,000","5,000",yearly,"180,000",married,non medical,15,traditional,retail agency,claim,death claim repudiated,misrepresentation
2,34,wife,self-employed,"350,000","5,000",yearly,"249,000",married,non medical,15,traditional,retail agency,claim,death claim repudiated,misrepresentation
3,34,wife,service,"325,000","5,000",yearly,"360,000",married,non medical,10,traditional,retail agency,claim,death claim repudiated,misrepresentation
4,54,husband,housewife,"100,000","5,000",yearly,"600,000",married,non medical,15,traditional,retail agency,claim,death claim repudiated,misrepresentation


In [ ]:
# Filter out rows where Fraud Category is 'Document Forgery' or 'Signature Forgery'
df = df[~df['Fraud Category'].isin(['document tampering', 'signature forgery'])]

In [ ]:
# Encoding categorical variables
categorical_cols = ["NOMINEE_RELATION", "OCCUPATION", "PREMIUMPAYMENTMODE", "HOLDERMARITALSTATUS", "INDIV_REQUIREMENTFLAG", "Product Type", "CHANNEL", "STATUS", "SUB_STATUS", "Fraud Category"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le  # Store encoder for inverse transform if needed

# Save all encoders
joblib.dump(label_encoders, "label_encoders.pkl")

# Check the class distribution
class_counts = df["Fraud Category"].value_counts()
print("Class Distribution:\n", class_counts)

# Define the classes to be merged
miscellaneous_classes = [3, 7, 1, 2, 4, 3, 0, 8, 5]
# Replace specified classes with a new label (assign a new numerical label, e.g., 11)
df["Fraud Category"] = df["Fraud Category"].replace(miscellaneous_classes, 11)

Class Distribution:
 Fraud Category
6    939
5     29
7     15
1     13
2      6
4      1
3      1
0      1
8      1
Name: count, dtype: int64


In [ ]:
print(df.size)
df = df.drop_duplicates()
print(df.size)

15090
14880


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# Select columns to normalize
cols_to_standardize = ["ASSURED_AGE", "POLICY SUMASSURED", "Premium", "Annual Income", "Policy Term"]

# Remove commas, trim spaces, and convert to float
df[cols_to_standardize] = df[cols_to_standardize].replace({",": ""}, regex=True).astype(float)

# Apply StandardScaler
scaler = StandardScaler()
df[cols_to_standardize] = scaler.fit_transform(df[cols_to_standardize])

# Save the trained scaler
joblib.dump(scaler, "scaler.pkl")

print(df.head())

   ASSURED_AGE  NOMINEE_RELATION  OCCUPATION  POLICY SUMASSURED   Premium  \
0    -3.344280                 6          10           0.572907  0.105021   
1    -1.800582                 5          10          -0.376039 -0.217042   
2    -1.681836                11           9          -0.376039 -0.217042   
3    -1.681836                11          10          -0.403949 -0.217042   
4     0.693084                 5           5          -0.655140 -0.217042   

   PREMIUMPAYMENTMODE  Annual Income  HOLDERMARITALSTATUS  \
0                   2      -0.040131                    2   
1                   4      -0.047462                    1   
2                   4      -0.045354                    1   
3                   4      -0.041964                    1   
4                   4      -0.034633                    1   

   INDIV_REQUIREMENTFLAG  Policy Term  Product Type  CHANNEL  STATUS  \
0                      1    -0.672248             4        2       1   
1                      1  

In [ ]:
for col, le in label_encoders.items():
    print(f"Column: {col}")
    print(dict(enumerate(le.classes_)))  # Mapping of encoded labels to original values
    print()

Column: NOMINEE_RELATION
{0: 'brother', 1: 'daughter', 2: 'father', 3: 'grand daughter', 4: 'grand son', 5: 'husband', 6: 'mother', 7: 'niece', 8: 'sister', 9: 'son', 10: 'spouse', 11: 'wife'}

Column: OCCUPATION
{0: 'agriculturist', 1: 'business', 2: 'construction labour', 3: 'defense retired', 4: 'family pension', 5: 'housewife', 6: 'other arm forces except police', 7: 'profession', 8: 'retired', 9: 'self-employed', 10: 'service', 11: 'student'}

Column: PREMIUMPAYMENTMODE
{0: 'half yearly', 1: 'monthly', 2: 'quarterly', 3: 'single', 4: 'yearly'}

Column: HOLDERMARITALSTATUS
{0: 'divorced', 1: 'married', 2: 'single', 3: 'widowed'}

Column: INDIV_REQUIREMENTFLAG
{0: 'medical', 1: 'non medical'}

Column: Product Type
{0: 'health', 1: 'non par', 2: 'pension', 3: 'traditional', 4: 'ulip', 5: 'variable'}

Column: CHANNEL
{0: 'bancassurance', 1: 'institutional alliance', 2: 'retail agency'}

Column: STATUS
{0: 'cancellation', 1: 'claim', 2: 'inforce', 3: 'lapse', 4: 'maturity', 5: 'rejecti

In [ ]:
# Feature-target split
X = df.drop("Fraud Category", axis=1)  # Replace with actual fraud category column
y = df["Fraud Category"]

In [ ]:
# Replace specified classes with a new label (assign a new numerical label, e.g., 11)
y = y.replace(miscellaneous_classes, 11)

# Check the class distribution
class_counts = y.value_counts()
print("Class Distribution:\n", class_counts)

Class Distribution:
 Fraud Category
6     925
11     38
5      29
Name: count, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train)
print(y_train)

     ASSURED_AGE  NOMINEE_RELATION  OCCUPATION  POLICY SUMASSURED   Premium  \
992     0.693084                11          10          -0.766781  0.609121   
606     0.218100                11          10          -0.208578 -0.158144   
968     1.049322                 5           8          -0.766781 -0.035006   
434    -0.375630                11          10          -0.208578 -0.091017   
560    -0.256884                11          10          -0.208578 -0.174460   
..           ...               ...         ...                ...       ...   
107     0.099354                11          10           1.577672  0.609121   
293    -0.256884                11          10          -0.208578 -0.091017   
895    -0.019392                11          10          -0.152757 -0.084716   
467     0.218100                11           1          -0.208578 -0.091017   
103    -0.256884                11          10           1.075289  0.207942   

     PREMIUMPAYMENTMODE  Annual Income  HOLDERMARIT

In [ ]:
from imblearn.over_sampling import SMOTE, RandomOverSampler # Import RandomOverSampler

# Handle class imbalance using SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
# Check the class distribution
class_counts = pd.Series(y_train).value_counts()
print("Class Distribution:\n", class_counts)

Class Distribution:
 Fraud Category
11    650
6     650
5     650
Name: count, dtype: int64


In [ ]:
# Convert labels to one-hot encoding (for 3 classes)
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_test = encoder.transform(y_test.values.reshape(-1, 1)) # Changed this line from the y_train version to prevent fitting to y_test

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define a simple neural network as an ANFIS approximation
model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),  # First fuzzy layer
    Dense(16, activation='relu'),  # Rule layer
    Dense(8, activation='relu'),  # Fuzzy aggregation layer
    Dense(y_train.shape[1], activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/1000


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4688 - loss: 1.0384 - val_accuracy: 0.7651 - val_loss: 0.8513
Epoch 2/1000
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6437 - loss: 0.7610 - val_accuracy: 0.7752 - val_loss: 0.8061
Epoch 3/1000
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6667 - loss: 0.6942 - val_accuracy: 0.8188 - val_loss: 0.6554
Epoch 4/1000
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6879 - loss: 0.6140 - val_accuracy: 0.7550 - val_loss: 0.7312
Epoch 5/1000
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7101 - loss: 0.5810 - val_accuracy: 0.7819 - val_loss: 0.6866
Epoch 6/1000
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7499 - loss: 0.5342 - val_accuracy: 0.7919 - val_loss: 0.6566
Epoch 7/1000
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8121 - loss: 0.4610 - val_accuracy: 0.7517 - val_loss: 0.7474
Epoch 8/1000
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8351 - loss: 0.4479 - val_accuracy: 0.8255 - val_lo

In [ ]:
from sklearn.metrics import classification_report

# Make predictions
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Classification report
print(classification_report(y_test_labels, y_pred_labels))


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
              precision    recall  f1-score   support

           0       0.43      0.33      0.38         9
           1       0.94      0.92      0.93       275
           2       0.05      0.07      0.06        14

    accuracy                           0.86       298
   macro avg       0.47      0.44      0.45       298
weighted avg       0.88      0.86      0.87       298



In [ ]:
joblib.dump(model, "fraud_classify_model.pkl")
print("Model saved successfully!")

Model saved successfully!


In [ ]:
rownum = 28

row = X_test.iloc[rownum].values
print(row)
print(row)
print(y_train[rownum])

y_preds = model.predict(row.reshape(1, -1))
print(y_preds)

[ 7.62875031e-03  1.20000000e+01  1.00000000e+01 -2.73697053e-01
 -3.11002681e-01  4.00000000e+00 -6.53488044e-02  1.00000000e+00
  1.00000000e+00  1.36818257e+00  3.00000000e+00  3.00000000e+00
  0.00000000e+00  1.10000000e+01]
[ 7.62875031e-03  1.20000000e+01  1.00000000e+01 -2.73697053e-01
 -3.11002681e-01  4.00000000e+00 -6.53488044e-02  1.00000000e+00
  1.00000000e+00  1.36818257e+00  3.00000000e+00  3.00000000e+00
  0.00000000e+00  1.10000000e+01]
2
[7]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
